In [1]:
import numpy as np
import cv2
from PIL import Image,ImageDraw
import math
import copy

In [2]:
sobel_X=np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
sobel_Y=np.array([[1,2,1],[0,0,0],[-1,-2,-1]])

image=Image.open('Data/bicycle.bmp')
img_arr=np.array(image)

In [3]:
def rgb2gray(image):
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    return gray

In [4]:
def display_image(arr,sigma):
    
    image_try=Image.fromarray(arr)
    ImageDraw.Draw(image_try).text((0, 0),f'sigma = {sigma}',(0, 0, 0))
    image_try.show()

In [5]:
def gaussian_smoothing(sigma=11,mean=0,ksize=5):
    
    n=(ksize-1)/2
    radius=int(sigma)
    X=np.linspace(-radius,radius,2*radius+1)
    Y=np.linspace(-radius,radius,2*radius+1)
    [grid_X,grid_Y]=np.meshgrid(X,Y)
    
    #gaussian_filter=np.exp(-(grid_X**2+grid_Y**2)/(2*(sigma)**2))/(math.sqrt(2*math.pi)*sigma)
    gaussian_filter=np.exp(-(grid_X**2+grid_Y**2)/(2*(sigma)**2))
    gaussian_filter=gaussian_filter/np.sum(gaussian_filter)
    
    return gaussian_filter

In [6]:
def add_padding(I,H):
    
    pad_R=int((H.shape[0]-1)/2)+(H.shape[0]+1)%2
    pad_C=int((H.shape[1]-1)/2)+(H.shape[1]+1)%2

    Z_R=np.zeros((pad_R,I.shape[1]))

    if H.shape[0]%2==0:
        Z_C=np.zeros((I.shape[0]+pad_R,pad_C))
    else:
        Z_C=np.zeros((I.shape[0]+2*pad_R,pad_C))

    if H.shape[0]%2==0 and H.shape[1]%2==0:
        I=np.append(Z_R,I,axis=0)
        I=np.append(Z_C,I,axis=1)


    elif H.shape[0]%2==0 and H.shape[1]%2==1:
        I=np.append(Z_R,I,axis=0)
        I=np.concatenate((Z_C,I,Z_C),axis=1)


    elif H.shape[0]%2==1 and H.shape[1]%2==0:
        I=np.concatenate((Z_R,I,Z_R),axis=0)
        I=np.append(Z_C,I,axis=1)


    else:
        I=np.concatenate((Z_R,I,Z_R),axis=0)
        I=np.concatenate((Z_C,I,Z_C),axis=1)
    
    return I

In [7]:
def convolution(img,H):
    
    I_pad=add_padding(img,H)
    
    conv=np.zeros(img.shape)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            window=I_pad[i:i+H.shape[0],j:j+H.shape[1]]
            conv[i,j]=np.sum(window*H)
    
    return conv

In [8]:
def harris_detector(I,alpha=0.04):
    
    I_gray=rgb2gray(I)

    for sigm in [1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]:
        gaussian_filter=gaussian_smoothing(sigma=sigm)
        Gaussian=convolution(I_gray,gaussian_filter)
        #Gaussian=cv2.GaussianBlur(I_gray,(3,3),3)

        grad_X=convolution(Gaussian,sobel_X)
        grad_X=convolution(grad_X,gaussian_smoothing(sigma=sigm))
        #grad_X=cv2.GaussianBlur(grad_X,(3,3),1.4)

        grad_Y=convolution(Gaussian,sobel_Y)
        grad_Y=convolution(grad_Y,gaussian_smoothing(sigma=sigm))
        #grad_Y=cv2.GaussianBlur(grad_Y,(3,3),1.4)
        grad_XX=np.square(grad_X)
        grad_YY=np.square(grad_Y)
        grad_XY=np.multiply(grad_X,grad_Y)

        max_R=0
        mat={}
        for row in range(1,I.shape[0]-1):
            for clm in range(1,I.shape[1]-1):

                Window_XX=np.sum(grad_XX[row-1:row+2,clm-1:clm+2])
                Window_XY=np.sum(grad_XY[row-1:row+2,clm-1:clm+2])
                Window_YY=np.sum(grad_YY[row-1:row+2,clm-1:clm+2])

                M=np.array([[Window_XX,Window_XY],[Window_XY,Window_YY]])

                R=np.linalg.det(M)-alpha*(np.trace(M))**2

                mat[(row,clm)]=R

                if R>max_R:
                    max_R=R

        mat_copy=mat.copy()
        print(max_R)
        for k,v in mat_copy.items():

            if not v>max_R*0.05:
                del mat[k]

        points=[]
        for k in mat.keys():
            max_R=0
            for i in range(k[0]-1,k[0]+3):
                for j in range(k[1]-1,k[1]+3):

                    if (i,j) in mat.keys():

                        if mat[(i,j)]>max_R:
                            pt=(i,j)
                            max_R=mat[(i,j)]
            if pt not in points:                
                points.append(pt)

        for val in points:
            I=cv2.circle(I,(val[1],val[0]),1,(255,0,0))

    #     for val in mat.keys():
    #         I=cv2.circle(I,(val[1],val[0]),1,(255,0,0))
        display_image(I,sigm)
    
    return mat,grad_X,grad_Y

In [9]:
I,x,y=harris_detector(img_arr)

444824138657.9731
408071812691.7141
382518064563.4314
363960337104.5098
350014408591.33075
339242039710.49927
330731258262.12274
323879510730.3629
318274753903.7368
313626897259.26984
43446461252.418304


In [ ]:
I_gray=rgb2gray(img_arr)
gaussian_filter=gaussian_smoothing(sigma=1.4)
#Gaussian=convolution(I_gray,gaussian_filter)
Gaussian=cv2.GaussianBlur(I_gray,(3,3),3)

grad_X=convolution(Gaussian,sobel_X)
#grad_X=convolution(grad_X,gaussian_smoothing(sigma=1.4))
grad_X=cv2.GaussianBlur(grad_X,(3,3),1.4)

grad_Y=convolution(Gaussian,sobel_Y)
#grad_Y=convolution(grad_Y,gaussian_smoothing(sigma=1.4))
grad_Y=cv2.GaussianBlur(grad_Y,(3,3),1.4)
grad_XX=np.square(grad_X)
grad_YY=np.square(grad_Y)
grad_XY=np.multiply(grad_X,grad_Y)

In [ ]:
grad_Y

In [ ]:
grad_Y=convolution(Gaussian,sobel_Y)
convolution(grad_Y,gaussian_smoothing(sigma=1.4,ksize=3))

In [ ]:
gaussian_smoothing(sigma=1.4,ksize=3)

In [ ]:
gaussian_filter=gaussian_smoothing(sigma=1,ksize=3)
Gaussian=convolution(I_gray,gaussian_filter)
image_try=Image.fromarray(Gaussian)
image_try.show()